In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

from sklearn.metrics import mean_absolute_percentage_error

In [200]:
train_data=pd.read_excel('Train.xlsx',na_values=['?','Unknown/Invalid','nan'])

In [201]:
x_test=pd.read_excel('Test.xlsx',na_values=['?','Unknown/Invalid','nan'])

In [202]:
y_test=pd.read_csv('samplesubmission.csv',na_values=['?','Unknown/Invalid','nan'])

In [203]:
x_train_data=train_data.drop('PropertyEvaluationvalue',axis=1)

In [204]:
y_train_data=train_data.PropertyEvaluationvalue

In [205]:
x_train_data.drop(['PropertyID','State','Address'],axis=1,inplace=True)

In [206]:
x_train_data.head()

,Borough,Surroundings,ZipCode,Block,Lot,BldgClassCategory,NoOfResidentialUnits,NoOfCommercialUnits,TotalNoOfUnits,LandAreaInSqFt,GrossAreaInSqFt,YearOfConstruction,TaxClass_AtEvaluationTime,BldgClass_AtEvaluationTime,DateOfEvaluation
0,1,BATHGATE,10456,2927,127,02 TWO FAMILY DWELLINGS,2,0,2,1900,2394,1990,1,B1,2014-05-30
1,1,BATHGATE,10457,3037,110,03 THREE FAMILY DWELLINGS,3,0,3,1687,3037,2002,1,C0,2014-09-13
2,1,BATHGATE,10457,2912,155,02 TWO FAMILY DWELLINGS,2,0,2,2000,2400,1989,1,B1,2014-09-28
3,1,BATHGATE,10457,2929,134,02 TWO FAMILY DWELLINGS,2,0,2,2356,2394,1991,1,B1,2014-06-05
4,1,BATHGATE,10457,3036,140,02 TWO FAMILY DWELLINGS,2,0,2,3346,2308,1993,1,B1,2015-02-07


In [207]:
x_test.drop(['PropertyID','State','Address'],axis=1,inplace=True)

In [208]:
y_test.drop(['PropertyID'],axis=1,inplace=True)

In [209]:
train_data.drop(train_data[train_data['LandAreaInSqFt']==5612000].index,axis=0,inplace=True)

In [210]:
train_data.drop(train_data[train_data['LandAreaInSqFt']==1806433].index,axis=0,inplace=True)

In [211]:
X_train=x_train_data
Y_train=y_train_data

In [212]:
X_test=x_test
Y_test=y_test.PropertyEvaluationvalue

In [213]:
num_cols = X_train[['ZipCode','LandAreaInSqFt','GrossAreaInSqFt']].columns
cat_cols = X_train[['BldgClassCategory','NoOfResidentialUnits','NoOfCommercialUnits','TaxClass_AtEvaluationTime',
       'BldgClass_AtEvaluationTime','Surroundings','Block','Lot','Borough',
        'TotalNoOfUnits','YearOfConstruction','DateOfEvaluation']].columns

In [214]:
X_train[cat_cols] = X_train[cat_cols].astype('category')

In [215]:
Numeric_transformer=Pipeline([('Missing values', SimpleImputer(strategy='mean'))
                              ,('scalar1',StandardScaler())])

In [216]:
categorical_transformer=Pipeline([('Missing values',SimpleImputer(strategy='most_frequent'))
                              ,('ohe',OneHotEncoder(handle_unknown='ignore'))])

In [217]:
preprocessor=(ColumnTransformer(transformers=[('num',Numeric_transformer,num_cols),('cat',categorical_transformer,cat_cols)]))

In [218]:
RF_pipe = Pipeline([('pp',preprocessor),('DT',DecisionTreeClassifier())])

In [219]:
RF_pipe.fit(X_train,Y_train)

Pipeline(steps=[('pp',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Missing '
                                                                   'values',
                                                                   SimpleImputer()),
                                                                  ('scalar1',
                                                                   StandardScaler())]),
                                                  Index(['ZipCode', 'LandAreaInSqFt', 'GrossAreaInSqFt'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Missing '
                                                                   'values',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['BldgClassCategory', 'NoOfResidentialUnits', 'NoOfCommercialUnits',
       'TaxClass_AtEvaluationTime', 'BldgClass_AtEvaluationTime',
       'Surroundings', 'Block', 'Lot', 'Borough', 'TotalNoOfUnits',
       'YearOfConstruction', 'DateOfEvaluation'],
      dtype='object'))])),
                ('DT', DecisionTreeClassifier())])

In [220]:
train_pred_RFPipe = RF_pipe.predict(X_train)
test_pred_RFPipe = RF_pipe.predict(X_test)

In [221]:
def evaluate_model(act, pred):
    from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred, average='weighted'))
    print("Precision: ", precision_score(act, pred, average='weighted'))

In [222]:
print("--Train--")
evaluate_model(Y_train, train_pred_RFPipe)
print("--Test--")
evaluate_model(Y_test, test_pred_RFPipe)

--Train--
Confusion Matrix 
 [[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Accurcay :  0.9932778108268887
Recall   :  0.9932778108268887
Precision:  0.9919726363920265
--Test--
Confusion Matrix 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accurcay :  0.0
Recall   :  0.0
Precision:  0.0


C:\Users\manoj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\manoj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\manoj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [223]:
mean_absolute_percentage_error(Y_test,test_pred_RFPipe)

0.5865900283521513